## augmentation 
- resize to a n * n dimension, using squished method (stretched)
- randomize: brightness, zoom, warp when rotated, and flip

## model
efnet b0, b5 then assemble

## training method
fastai with efficient + progressive training + fp16
then progressively trained from 128, 256, 368, 468 with 64,64,64,32,16 batch size respectively

## loss function 
FocalLoss -> label smoothing cross entropy


In [1]:
import warnings
warnings.filterwarnings("ignore")

from fastai.vision import *
from fastai.metrics import error_rate
from fastai.callbacks import *
from fastai.vision.models.efficientnet import *

import torch

torch.cuda.empty_cache()

In [2]:
#masukkin image dataset nya disini, tapi per folder harus ada labelnya
#Reference Implementation fastai --->>> https://gilberttanner.com/blog/fastai-image-classification
defaults.device = torch.device("cuda")

In [3]:
def get_data(bs, size): 
    tfms = get_transforms(max_lighting=0.4, max_zoom=1.2, max_warp=0.2, max_rotate=20, xtra_tfms=[flip_lr()])
    return ImageDataBunch.from_folder(Path('./dataset'),
                                  train = 'train/',
                                  valid_pct = 0.1,
                                  resize_method=ResizeMethod.SQUISH, 
                                  ds_tfms = tfms,
                                  size = size,
                                  bs = bs,
                                  num_workers = 50
                                  ).normalize(imagenet_stats)

# 1. 128

In [4]:
data = get_data(64, 128)

len(data.classes), data.c

(42, 42)

In [5]:
model = EfficientNet.from_pretrained('efficientnet-b0')
# model

Loaded pretrained weights for efficientnet-b0


In [6]:
model._fc = nn.Linear(model._fc.in_features, data.c)

In [ ]:
from efficientnet_pytorch import EfficientNet

def get_model(pretrained=True, **kwargs): 
    model = EfficientNet.from_pretrained('efficientnet-b0')
    model._fc = nn.Linear(model._fc.in_features, data.c) # check the top most layer
    return model 

def stage1(learn, data=None, epoch = 5):
    if data: 
        learn.data=data
        learn.to_fp16()
        
    learn.freeze()
    learn.fit_one_cycle(epoch)
    
def stage2(learn, load_filename=None, epoch=5):
    if load_filename: 
        learn.load(load_filename, purge=True)
    learn.unfreeze()
    learn.lr_find()
    learn.recorder.plot(suggestion=True)
    min_grad_lr = learn.recorder.min_grad_lr
    learn.fit_one_cycle(epoch, slice(min_grad_lr/40, min_grad_lr))

learn = Learner(data, model,
               metrics = [accuracy], 
                bn_wd=False, #disable weight decay 
                loss_func = LabelSmoothingCrossEntropy(), 
                callback_fns=[BnFreeze,
                             partial(SaveModelCallback, monitor='accuracy', name='most_accurate')],
               path='./dataset').to_fp16() # because different clf layer, we use Learner.
learn.split(lambda m: (model._conv_head,))

In [8]:
data.c

42

In [ ]:
stage1(learn)


In [ ]:
stage2(learn)

In [ ]:
learn.save('b5-epoch5-128')
learn.load('b5-epoch5-128', purge=True)



# 2. 256

In [ ]:
data = get_data(64, 256)
learn.load('b5-epoch5-256', purge=True)
learn.to_fp16()
stage1(learn, data)
stage2(learn)

In [ ]:
learn.save('b5-epoch5-256')
learn.load('b5-epoch5-256', purge=True)


# 3. 368

In [ ]:
# SINI
data = get_data(32, 368)
learn.load('b0-epoch5-384', purge=True)
learn.to_fp16()
stage1(learn, data)
stage2(learn)

In [ ]:
learn.save('b5-epoch5-368')

# 4. 468

In [ ]:
# SINI
data = get_data(32, 368)
learn.load('b0-epoch5-384', purge=True)
learn.to_fp16()
stage1(learn, data)
stage2(learn)

In [ ]:
learn.save('b5-epoch5-468')

# 5. 512

In [ ]:
# learn.load('b5-epoch5-468', purge=True)
data = get_data(16, 512)
learn.load('b5-epoch5-468', purge=True)
learn.data=data
learn.to_fp16()
stage1(learn, data)
stage2(learn,epoch=6)

In [ ]:
learn.save('b0 final ')
learn.export('b0')

In [ ]:
data = get_data(16, 512)
learn.load('b0 final ', purge=True)
learn.data = data
learn.to_fp16()
stage1(learn,epoch=6)
learn.save('b0 final-1')

In [ ]:
import os
os.system('python finish_me.py')

In [ ]:
# learn.load('b5-epoch5-468', purge=True)
# stage1(learn, data)
learn.load('b0 final-1')
data = get_data(16, 512)

learn.data=data
learn.to_fp16()

# stage2(learn,epoch=5)

In [ ]:
import os
os.system('python finish_me.py')

In [ ]:
learn.model

In [ ]:
import pandas as pd

In [9]:
learn.load('b0 final-1')
data = get_data(16, 512)

learn.data=data
learn.to_fp32()


learn.export()

In [10]:
learn = load_learner('./dataset')

In [11]:
image_test = open_image('./bag.jpeg')

In [ ]:
type(data.train_ds[0][0])

In [ ]:
data.train_ds[1]

In [15]:
hasil_prediksi = learn.predict(image_test)

In [25]:
str(hasil_prediksi[0])

'09'

In [18]:
dir(hasil_prediksi[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'apply_tfms',
 'data',
 'obj',
 'show']

In [16]:
print(hasil_prediksi) # ini kenapa ada 3 prediksi anjir fak
"""
It returns a tuple of three things: 
the object predicted (with the class in this instance), 
the underlying data (here the corresponding index)  
the raw probabilities. 
"""


(Category tensor(9), tensor(9), tensor([ 8.7969,  8.5134,  9.4475, 10.5954,  7.6278,  7.9949,  8.6377,  8.9289,
        11.5912, 16.6772,  8.0053,  8.6517,  7.4771,  8.9308,  8.0464,  8.9818,
         8.7663,  8.5720,  8.4165,  8.9928,  7.5449,  9.9747, 10.0802,  9.8516,
         8.9836,  8.9228,  8.3149,  9.1229,  8.6906,  6.4960,  6.9768,  8.8049,
         6.2074,  7.8944,  7.4179,  8.3689,  9.2012, 11.5521,  6.6051, 10.1046,
         7.0696,  7.6125]))


In [ ]:
max(hasil_prediksi[2])

In [ ]:
def make_submission(learn, filename):
    log_preds, test_labels = learn.get_preds(ds_type=DatasetType.Test)
    preds = np.argmax(log_preds, 1)
    a = np.array(preds)
    submission = pd.DataFrame({'image_name': os.listdir('data/test'), 'label': a})
    submission.to_csv(path/filename, index=False)